# GPU-Accelerated Sorting Algorithm Analysis

This notebook performs sorting algorithm analysis using GPU acceleration with CuPy for better performance on Google Colab.

**Requirements:** 
- GPU runtime in Google Colab
- CuPy for GPU-accelerated array operations

In [ ]:
# Install required packages for GPU acceleration
!pip install cupy-cuda12x matplotlib seaborn numpy

# Check if GPU is available
!nvidia-smi

In [ ]:
import cupy as cp
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Check GPU availability
print(f"CuPy version: {cp.__version__}")
print(f"GPU available: {cp.cuda.is_available()}")
if cp.cuda.is_available():
    print(f"GPU name: {cp.cuda.Device().name.decode()}")
    print(f"GPU memory: {cp.cuda.Device().mem_info[1] / 1e9:.1f} GB")

# Test sizes - larger sizes for GPU testing
sizes = [1000, 5000, 10000, 25000, 50000, 100000, 250000, 500000]

In [ ]:
def time_sort_cpu(sort_func, arr, runs=5):
    """Time a CPU sorting function"""
    times = []
    for _ in range(runs):
        test_arr = arr.copy()
        start = time.perf_counter()
        sort_func(test_arr)
        end = time.perf_counter()
        times.append(end - start)
    return np.mean(times)

def time_sort_gpu(sort_func, arr, runs=5):
    """Time a GPU sorting function"""
    times = []
    gpu_arr = cp.array(arr)
    
    for _ in range(runs):
        test_arr = gpu_arr.copy()
        cp.cuda.Stream.null.synchronize()  # Ensure GPU is ready
        start = time.perf_counter()
        sort_func(test_arr)
        cp.cuda.Stream.null.synchronize()  # Wait for completion
        end = time.perf_counter()
        times.append(end - start)
    
    return np.mean(times)

def generate_test_data(size, data_type='random'):
    """Generate test data of different types"""
    if data_type == 'random':
        return np.random.randint(0, 100000, size)
    elif data_type == 'sorted':
        return np.arange(size)
    elif data_type == 'reverse':
        return np.arange(size, 0, -1)
    elif data_type == 'nearly_sorted':
        arr = np.arange(size)
        # Swap 5% of elements randomly
        swaps = size // 20
        for _ in range(swaps):
            i, j = np.random.randint(0, size, 2)
            arr[i], arr[j] = arr[j], arr[i]
        return arr

## CPU Sorting Algorithms

In [ ]:
def bubble_sort_cpu(arr):
    """Optimized Bubble Sort with early termination"""
    n = len(arr)
    for i in range(n):
        swapped = False
        for j in range(0, n - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
        if not swapped:
            break

def insertion_sort_cpu(arr):
    """Optimized Insertion Sort"""
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > key:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key

def quick_sort_cpu(arr):
    """Optimized Quick Sort with random pivot"""
    def partition(arr, low, high):
        # Random pivot selection
        random_idx = np.random.randint(low, high + 1)
        arr[random_idx], arr[high] = arr[high], arr[random_idx]
        
        pivot = arr[high]
        i = low - 1
        for j in range(low, high):
            if arr[j] <= pivot:
                i += 1
                arr[i], arr[j] = arr[j], arr[i]
        arr[i + 1], arr[high] = arr[high], arr[i + 1]
        return i + 1

    def quick_sort_helper(arr, low, high):
        if low < high:
            pi = partition(arr, low, high)
            quick_sort_helper(arr, low, pi - 1)
            quick_sort_helper(arr, pi + 1, high)

    quick_sort_helper(arr, 0, len(arr) - 1)

def merge_sort_cpu(arr):
    """Optimized Merge Sort"""
    if len(arr) <= 1:
        return arr
    
    mid = len(arr) // 2
    left = merge_sort_cpu(arr[:mid])
    right = merge_sort_cpu(arr[mid:])
    
    # Merge
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    
    result.extend(left[i:])
    result.extend(right[j:])
    
    # Copy back to original array
    for i, val in enumerate(result):
        arr[i] = val

## GPU-Accelerated Sorting Algorithms

In [ ]:
def radix_sort_gpu(arr):
    """GPU-accelerated Radix Sort using CuPy"""
    if len(arr) == 0:
        return
    
    max_val = cp.max(arr)
    exp = 1
    
    while max_val // exp > 0:
        # Count sort based on digit represented by exp
        n = len(arr)
        output = cp.zeros(n, dtype=arr.dtype)
        count = cp.zeros(10, dtype=cp.int32)
        
        # Count occurrences of each digit
        digits = (arr // exp) % 10
        for i in range(10):
            count[i] = cp.sum(digits == i)
        
        # Cumulative count
        for i in range(1, 10):
            count[i] += count[i - 1]
        
        # Build output array
        for i in range(n - 1, -1, -1):
            digit = (arr[i] // exp) % 10
            count[digit] -= 1
            output[count[digit]] = arr[i]
        
        # Copy output to arr
        arr[:] = output[:]
        exp *= 10

def gpu_built_in_sort(arr):
    """Use CuPy's built-in GPU sort"""
    arr[:] = cp.sort(arr)

def bitonic_sort_gpu(arr):
    """GPU Bitonic Sort (works best with power of 2 sizes)"""
    n = len(arr)
    
    # Pad to next power of 2 if necessary
    next_pow2 = 1
    while next_pow2 < n:
        next_pow2 *= 2
    
    if next_pow2 > n:
        padded = cp.full(next_pow2, cp.max(arr) + 1, dtype=arr.dtype)
        padded[:n] = arr
        arr_to_sort = padded
    else:
        arr_to_sort = arr
    
    def bitonic_merge(arr, start, cnt, up):
        if cnt > 1:
            k = cnt // 2
            for i in range(start, start + k):
                if (arr[i] > arr[i + k]) == up:
                    arr[i], arr[i + k] = arr[i + k], arr[i]
            bitonic_merge(arr, start, k, up)
            bitonic_merge(arr, start + k, k, up)
    
    def bitonic_sort_recursive(arr, start, cnt, up):
        if cnt > 1:
            k = cnt // 2
            bitonic_sort_recursive(arr, start, k, True)
            bitonic_sort_recursive(arr, start + k, k, False)
            bitonic_merge(arr, start, cnt, up)
    
    bitonic_sort_recursive(arr_to_sort, 0, len(arr_to_sort), True)
    
    if next_pow2 > n:
        arr[:] = arr_to_sort[:n]

## Performance Testing

In [ ]:
# Test different data types
data_types = ['random', 'sorted', 'reverse', 'nearly_sorted']

# CPU algorithms
cpu_algorithms = {
    'Bubble Sort (CPU)': bubble_sort_cpu,
    'Insertion Sort (CPU)': insertion_sort_cpu,
    'Quick Sort (CPU)': quick_sort_cpu,
    'Merge Sort (CPU)': merge_sort_cpu,
    'NumPy Sort (CPU)': lambda arr: arr.sort()
}

# GPU algorithms (if GPU is available)
gpu_algorithms = {}
if cp.cuda.is_available():
    gpu_algorithms = {
        'Radix Sort (GPU)': radix_sort_gpu,
        'CuPy Sort (GPU)': gpu_built_in_sort,
        'Bitonic Sort (GPU)': bitonic_sort_gpu
    }

print(f"Testing {len(cpu_algorithms)} CPU algorithms and {len(gpu_algorithms)} GPU algorithms")
print(f"Data types: {data_types}")
print(f"Array sizes: {sizes}")

In [ ]:
# Run comprehensive performance tests
results = {}

for data_type in ['random']:  # Focus on random data for main analysis
    print(f"\nTesting with {data_type} data...")
    results[data_type] = {}
    
    for alg_name, alg_func in {**cpu_algorithms, **gpu_algorithms}.items():
        if 'Bubble' in alg_name and max(sizes) > 50000:
            # Skip bubble sort for very large sizes
            test_sizes = [s for s in sizes if s <= 50000]
        else:
            test_sizes = sizes
            
        results[data_type][alg_name] = []
        print(f"  Testing {alg_name}...")
        
        for size in test_sizes:
            arr = generate_test_data(size, data_type)
            
            try:
                if 'GPU' in alg_name:
                    avg_time = time_sort_gpu(alg_func, arr)
                else:
                    avg_time = time_sort_cpu(alg_func, arr)
                
                results[data_type][alg_name].append(avg_time)
                print(f"    Size {size:>6}: {avg_time:.6f}s")
                
            except Exception as e:
                print(f"    Size {size:>6}: ERROR - {e}")
                results[data_type][alg_name].append(float('inf'))
        
        # Pad with inf for skipped sizes
        while len(results[data_type][alg_name]) < len(sizes):
            results[data_type][alg_name].append(float('inf'))

print("\nPerformance testing completed!")

## Visualization and Analysis

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('GPU-Accelerated Sorting Algorithm Performance Analysis', fontsize=16)

data_type = 'random'
colors = plt.cm.tab10(np.linspace(0, 1, len(results[data_type])))

# 1. Linear scale comparison
ax1 = axes[0, 0]
for i, (alg_name, times) in enumerate(results[data_type].items()):
    valid_times = [t if t != float('inf') else np.nan for t in times]
    valid_sizes = sizes[:len(valid_times)]
    ax1.plot(valid_sizes, valid_times, 'o-', label=alg_name, color=colors[i], linewidth=2, markersize=6)

ax1.set_xlabel('Input Size')
ax1.set_ylabel('Time (seconds)')
ax1.set_title('Performance Comparison (Linear Scale)')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# 2. Log scale comparison
ax2 = axes[0, 1]
for i, (alg_name, times) in enumerate(results[data_type].items()):
    valid_times = [t if t != float('inf') else np.nan for t in times]
    valid_sizes = sizes[:len(valid_times)]
    ax2.semilogy(valid_sizes, valid_times, 'o-', label=alg_name, color=colors[i], linewidth=2, markersize=6)

ax2.set_xlabel('Input Size')
ax2.set_ylabel('Time (seconds) - Log Scale')
ax2.set_title('Performance Comparison (Log Scale)')
ax2.grid(True, alpha=0.3)

# 3. GPU vs CPU comparison for largest size
ax3 = axes[0, 2]
gpu_times = []
cpu_times = []
labels = []

for alg_name, times in results[data_type].items():
    if times[-1] != float('inf'):
        if 'GPU' in alg_name:
            gpu_times.append(times[-1])
            labels.append(alg_name.replace(' (GPU)', ''))
        else:
            cpu_times.append(times[-1])

x_pos = np.arange(len(labels))
if gpu_times and cpu_times:
    width = 0.35
    bars1 = ax3.bar(x_pos - width/2, cpu_times[:len(gpu_times)], width, label='CPU', alpha=0.8)
    bars2 = ax3.bar(x_pos + width/2, gpu_times, width, label='GPU', alpha=0.8)
    
    ax3.set_xlabel('Algorithm')
    ax3.set_ylabel('Time (seconds)')
    ax3.set_title(f'GPU vs CPU at Size {sizes[-1]}')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(labels, rotation=45)
    ax3.legend()
    ax3.grid(True, alpha=0.3)

# 4. Speedup analysis
ax4 = axes[1, 0]
if cp.cuda.is_available() and 'CuPy Sort (GPU)' in results[data_type] and 'NumPy Sort (CPU)' in results[data_type]:
    gpu_times = results[data_type]['CuPy Sort (GPU)']
    cpu_times = results[data_type]['NumPy Sort (CPU)']
    speedup = [cpu/gpu if gpu != float('inf') and gpu > 0 else 0 for cpu, gpu in zip(cpu_times, gpu_times)]
    
    ax4.plot(sizes, speedup, 'ro-', linewidth=3, markersize=8)
    ax4.set_xlabel('Input Size')
    ax4.set_ylabel('Speedup (CPU time / GPU time)')
    ax4.set_title('GPU Speedup vs CPU')
    ax4.grid(True, alpha=0.3)
    ax4.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='No speedup')
    ax4.legend()

# 5. Memory efficiency (theoretical)
ax5 = axes[1, 1]
memory_usage = {}
for size in sizes:
    memory_usage[size] = {
        'Quick Sort': size * 8 * np.log2(size),  # O(log n) space
        'Merge Sort': size * 8 * 2,  # O(n) space
        'Radix Sort': size * 8 * 2,  # O(n) space
        'In-place': size * 8  # O(1) additional space
    }

for alg, color in zip(['Quick Sort', 'Merge Sort', 'Radix Sort', 'In-place'], ['blue', 'green', 'red', 'orange']):
    memory_vals = [memory_usage[size][alg] / 1e6 for size in sizes]  # Convert to MB
    ax5.plot(sizes, memory_vals, 'o-', label=alg, color=color, linewidth=2)

ax5.set_xlabel('Input Size')
ax5.set_ylabel('Memory Usage (MB)')
ax5.set_title('Theoretical Memory Usage')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Algorithm complexity comparison
ax6 = axes[1, 2]
n_vals = np.array(sizes)
complexities = {
    'O(n²)': n_vals**2,
    'O(n log n)': n_vals * np.log2(n_vals),
    'O(n)': n_vals,
    'O(log n)': np.log2(n_vals)
}

for complexity, values in complexities.items():
    normalized_values = values / values[0]  # Normalize to first value
    ax6.loglog(sizes, normalized_values, 'o-', label=complexity, linewidth=2)

ax6.set_xlabel('Input Size')
ax6.set_ylabel('Normalized Operations')
ax6.set_title('Time Complexity Comparison')
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Performance summary table
print("\n" + "="*80)
print("PERFORMANCE SUMMARY")
print("="*80)

data_type = 'random'
print(f"\nData Type: {data_type.upper()}")
print("-" * 80)
print(f"{'Algorithm':<20} {'Smallest':<12} {'Largest':<12} {'Complexity':<15} {'Type':<8}")
print("-" * 80)

complexity_map = {
    'Bubble Sort (CPU)': 'O(n²)',
    'Insertion Sort (CPU)': 'O(n²)',
    'Quick Sort (CPU)': 'O(n log n)',
    'Merge Sort (CPU)': 'O(n log n)',
    'NumPy Sort (CPU)': 'O(n log n)',
    'Radix Sort (GPU)': 'O(d × n)',
    'CuPy Sort (GPU)': 'O(n log n)',
    'Bitonic Sort (GPU)': 'O(log²n)'
}

for alg_name, times in results[data_type].items():
    if times[0] != float('inf') and times[-1] != float('inf'):
        smallest_time = times[0]
        largest_time = times[-1]
        complexity = complexity_map.get(alg_name, 'Unknown')
        alg_type = 'GPU' if 'GPU' in alg_name else 'CPU'
        
        print(f"{alg_name:<20} {smallest_time:<12.6f} {largest_time:<12.6f} {complexity:<15} {alg_type:<8}")

# GPU Speedup Analysis
if cp.cuda.is_available():
    print("\n" + "="*80)
    print("GPU SPEEDUP ANALYSIS")
    print("="*80)
    
    if 'CuPy Sort (GPU)' in results[data_type] and 'NumPy Sort (CPU)' in results[data_type]:
        gpu_times = results[data_type]['CuPy Sort (GPU)']
        cpu_times = results[data_type]['NumPy Sort (CPU)']
        
        print(f"\n{'Size':<10} {'CPU Time':<12} {'GPU Time':<12} {'Speedup':<10} {'Efficiency':<12}")
        print("-" * 60)
        
        for i, size in enumerate(sizes):
            if gpu_times[i] != float('inf') and cpu_times[i] != float('inf') and gpu_times[i] > 0:
                speedup = cpu_times[i] / gpu_times[i]
                efficiency = speedup * 100  # Percentage
                print(f"{size:<10} {cpu_times[i]:<12.6f} {gpu_times[i]:<12.6f} {speedup:<10.2f}x {efficiency:<12.1f}%")

    print("\nNotes:")
    print("- Speedup = CPU Time / GPU Time")
    print("- Efficiency shows percentage improvement")
    print("- GPU performance improves with larger datasets due to parallelization")
    print("- Memory transfer overhead affects small dataset performance")

# Best algorithm recommendations
print("\n" + "="*80)
print("ALGORITHM RECOMMENDATIONS")
print("="*80)
print("\nBest algorithms by use case:")
print("• Small datasets (< 10,000): NumPy Sort (CPU) or Insertion Sort")
print("• Medium datasets (10,000 - 100,000): Quick Sort (CPU) or CuPy Sort (GPU)")
print("• Large datasets (> 100,000): CuPy Sort (GPU) or Radix Sort (GPU)")
print("• Nearly sorted data: Insertion Sort or optimized Bubble Sort")
print("• Memory constrained: Quick Sort (in-place) or Bubble Sort")
print("• Parallel processing: GPU-based algorithms (CuPy, Radix)")

print("\nComplexity Summary:")
print("• O(n²): Bubble Sort, Insertion Sort - Good for small/nearly sorted data")
print("• O(n log n): Quick Sort, Merge Sort, NumPy/CuPy Sort - General purpose")
print("• O(d × n): Radix Sort - Excellent for integers with limited range")
print("• O(log²n): Bitonic Sort - Highly parallel, good for GPU")

## Additional GPU Performance Tests

In [ ]:
# Test with different data distributions if GPU is available
if cp.cuda.is_available():
    print("Testing GPU algorithms with different data distributions...\n")
    
    test_size = 100000
    distributions = {
        'Random': np.random.randint(0, 100000, test_size),
        'Sorted': np.arange(test_size),
        'Reverse Sorted': np.arange(test_size, 0, -1),
        'Nearly Sorted': np.concatenate([np.arange(test_size-1000), np.random.randint(0, 1000, 1000)]),
        'Many Duplicates': np.random.choice(100, test_size),
        'Normal Distribution': np.random.normal(50000, 15000, test_size).astype(int)
    }
    
    gpu_alg_results = {}
    
    for dist_name, data in distributions.items():
        gpu_alg_results[dist_name] = {}
        print(f"Distribution: {dist_name}")
        
        for alg_name, alg_func in gpu_algorithms.items():
            try:
                avg_time = time_sort_gpu(alg_func, data, runs=3)
                gpu_alg_results[dist_name][alg_name] = avg_time
                print(f"  {alg_name}: {avg_time:.6f}s")
            except Exception as e:
                print(f"  {alg_name}: ERROR - {e}")
                gpu_alg_results[dist_name][alg_name] = float('inf')
        print()
    
    # Visualize results
    if gpu_alg_results:
        fig, ax = plt.subplots(figsize=(12, 8))
        
        distributions_list = list(gpu_alg_results.keys())
        algorithms_list = list(gpu_algorithms.keys())
        
        x = np.arange(len(distributions_list))
        width = 0.25
        
        for i, alg in enumerate(algorithms_list):
            times = [gpu_alg_results[dist].get(alg, float('inf')) for dist in distributions_list]
            times = [t if t != float('inf') else 0 for t in times]  # Replace inf with 0 for plotting
            ax.bar(x + i * width, times, width, label=alg)
        
        ax.set_xlabel('Data Distribution')
        ax.set_ylabel('Time (seconds)')
        ax.set_title('GPU Algorithm Performance by Data Distribution')
        ax.set_xticks(x + width)
        ax.set_xticklabels(distributions_list, rotation=45)
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
else:
    print("GPU not available for additional testing.")

## Conclusion and Key Findings

This notebook demonstrates the power of GPU acceleration for sorting algorithms:

### Key Findings:
1. **GPU algorithms excel with large datasets** due to parallel processing capabilities
2. **Memory transfer overhead** affects performance on small datasets
3. **CuPy's built-in sort** typically provides the best balance of performance and reliability
4. **Radix sort** can be extremely fast for integer data with limited range
5. **Different data distributions** can significantly impact algorithm performance

### Recommendations:
- Use **CPU algorithms** for small datasets (< 10,000 elements)
- Use **GPU algorithms** for large datasets (> 100,000 elements)
- **CuPy Sort** is generally the best choice for GPU sorting
- Consider **Radix Sort** for integer-only data
- Always profile with your specific data characteristics

### Google Colab Optimization Tips:
1. Enable GPU runtime: Runtime → Change runtime type → GPU
2. Monitor GPU memory usage with `!nvidia-smi`
3. Use `cp.cuda.Device().mem_info` to check available memory
4. Clear GPU memory between tests with `cp.get_default_memory_pool().free_all_blocks()`